In [71]:
%matplotlib inline
import sklearn
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import math
import nltk

import snowballstemmer

from keras import models
from keras import layers

from nltk.stem import SnowballStemmer

from nltk.corpus import stopwords

from tokenizer import tokenizer

from sklearn.linear_model import *
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import *
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

from functions import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
tweet_pos = pd.read_csv('data/train_pos.txt', header = None, sep = "\r\n", engine = 'python')
tweet_neg = pd.read_csv('data/train_neg.txt', header = None, sep = "\r\n", engine = 'python')
tweet_pos['pred'] = 1
tweet_neg['pred'] = 0

tweet_pos.columns = ['tweet', 'pred']
tweet_neg.columns = ['tweet', 'pred']

all_tweets = tweet_neg.append(tweet_pos)
all_tweets = all_tweets.reset_index().drop(['index'], axis = 1)

all_tweets.head()

,tweet,pred
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...,0
1,glad i dot have taks tomorrow ! ! #thankful #s...,0
2,1-3 vs celtics in the regular season = were fu...,0
3,<user> i could actually kill that girl i'm so ...,0
4,<user> <user> <user> i find that very hard to ...,0


In [107]:
def tokenize(t):
    tweet_tok = TweetTokenizer()
    tokens = tweet_tok.tokenize(t)
    wnl = WordNetLemmatizer()
    stems = []
    for item in tokens:
        stems.append(wnl.lemmatize(item))
    return stems

In [108]:
#pd.read_csv(PATH + 'common-english-prep-conj.txt').values.flatten().tolist()  + \
#pd.read_csv(PATH + 'punctuation.txt', delimiter='\n').values.flatten().tolist() 
#pd.read_csv(PATH + 'twitter-stopwords - TA.txt.txt').values.flatten().tolist() 

# Stop words for tweets
PATH = 'data/stopwords/'
stopw = pd.read_csv(PATH + 'twitter-stopwords.txt').values.flatten().tolist()

In [109]:
tweet_test = pd.read_csv('data/test_data.txt', header = None, sep = "\r\n", engine = 'python')
tweet_clean = clean_data(tweet_test.values)
np.reshape(tweet_clean, (10000,))
tweet_TE = tweet_clean.flatten()

In [110]:
vectorizer = TfidfVectorizer(analyzer='word', stop_words = stopw, tokenizer=tokenize, ngram_range=(1,2))

In [111]:
X = vectorizer.fit_transform(all_tweets.values[:, 0])
Y = all_tweets.values[:, 1].astype(int)

In [112]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

# Define classifiers

In [113]:
clf = LinearSVC()

# Find out the best parameters

In [114]:
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

In [115]:
accuracy_score(y_pred, y_test)

0.848525

In [106]:
accuracy_score(y_pred, y_test)

0.84875

In [95]:
accuracy_score(y_pred, y_test)

0.845975

In [85]:
accuracy_score(y_pred, y_test)

0.84735

In [68]:
accuracy_score(y_pred, y_test)

0.851625

# Take the average

In [20]:
def class_by_threshold(array, threshold):
    ret_array = np.ones(array.shape[0])
    for i in range(len(ret_array)):
        if(array[i, 0] >= threshold): ret_array[i] = 0
    return ret_array

In [21]:
from ipywidgets import FloatSlider, interact

#Use interact to choose the threshold and display the confusion matrix and the metrics
def print_result(Threshold):
    y_pred = class_by_threshold(y_pred_proba, Threshold)
    print('Accuracy = %.4f' % accuracy_score(y_pred, y_test))
    
interact(print_result, Threshold = FloatSlider(min=0, max=1, step=0.01, readout_format='.2f'))

interactive(children=(FloatSlider(value=0.0, description='Threshold', max=1.0, step=0.01), Output()), _dom_classes=('widget-interact',))

<function __main__.print_result>

# Keep track of scores per classifier (small dataset)

In [51]:
# Logistic regression, small dataset
accuracy_score(y_test, y_pred)

0.84425

In [44]:
# Logistic regression, small dataset
accuracy_score(y_test, y_pred)

0.8447

# Keep track of scores per classifier (full dataset)

In [95]:
# Logistic regression
accuracy_score(y_test, y_pred)

0.865242

In [98]:
# LinearSVC
accuracy_score(y_test, y_pred)

0.867602

In [65]:
accuracy_score(y_test, y_pred)

0.85035

# Output the real predicitons

In [49]:
X_TE = vectorizer.transform(tweet_TE)

In [50]:
y_pred_TE = model.predict(X_TE)

In [51]:
build_submission(y_pred_TE, '12')